In [34]:
from dash import Dash, dash_table, dcc, html, Output, Input
import pandas as pd
import plotly.graph_objs as go
import requests

In [35]:
# # you can get the chain list straihgt from the API
# url = 'https://api.llama.fi/v2/chains'
# response = requests.get(url)
# if response.status_code == 200: # data was fetched successfully
#     data = response.json()
#     df = pd.DataFrame(data)
# else:
#     raise Exception('Error fetching data from DeFiLlama API')

# df = df[['name', 'tvl']]

# # Sort the data to get the top 10 chains by TVL
# df = df.sort_values(by='tvl', ascending=False).head(10)


In [36]:
# you can get the chain list straihgt from the API
url = 'https://api.llama.fi/v2/chains'
response = requests.get(url)
if response.status_code == 200: # data was fetched successfully
    data = response.json()
    df = pd.DataFrame(data)
else:
    raise Exception('Error fetching data from DeFiLlama API')

dex_all_chains_url = "https://api.llama.fi/overview/dexs?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyVolume"
dex_all_chains_response = requests.get(dex_all_chains_url)

dex_all_chains_data = dex_all_chains_response.json()
dex_all_chains = set(dex_all_chains_data.get('allChains', []))


fees_all_chains_url = "https://api.llama.fi/overview/fees?excludeTotalDataChart=true&excludeTotalDataChartBreakdown=true&dataType=dailyFees"
fees_all_chains_response = requests.get(fees_all_chains_url)

fees_all_chains_data = fees_all_chains_response.json()
fees_all_chains = set(fees_all_chains_data.get('allChains', []))

common_chains = dex_all_chains.intersection(fees_all_chains)

df = df[['name', 'tvl']]
df = df[df['name'].isin(common_chains)]

# Sort the data to get the top 10 chains by TVL
df = df.sort_values(by='tvl', ascending=False).head(10)

In [37]:
def fetch_historical_data(chain_name, metric):
    if metric == 'tvl':
        url = f'https://api.llama.fi/v2/historicalChainTvl/{chain_name}'
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data)  # Directly create DataFrame from the list of dictionaries
            df['date'] = pd.to_datetime(df['date'], unit='s')
            print(df.tail(120))
            return df[['date', metric]].tail(120)  # Get the last 120 days of the specified metric
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'volume':
        url = (f'https://api.llama.fi/overview/dexs/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyVolume')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            print(df.tail(120))
            return df[['date', metric]].tail(120)
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
        
    elif metric == 'fee':
        url = (f'https://api.llama.fi/overview/fees/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyFees')
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', metric])   
            df['date'] = pd.to_datetime(df['timestamp'], unit='s')
            df = df.sort_values(by='date')
            print(df.tail(120))
            return df[['date', metric]].tail(120)
        else:
            raise Exception(f'Error fetching historical {metric} data for {chain_name}')
    else:
        raise ValueError('Invalid metric specified')

In [41]:
# list_of_volume = []
# for chain_name in df['name']:
#     historical_data_df = fetch_historical_data(chain_name, 'fee')
#     list_of_volume.append(historical_data_df)

# data of bitcoin
chain_name = 'Bitcoin'
url = (f'https://api.llama.fi/overview/dexs/{chain_name}'
                  '?excludeTotalDataChart=false'
                  '&excludeTotalDataChartBreakdown=true'
                  '&dataType=dailyVolume')
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    df = pd.DataFrame(data['totalDataChart'], columns=['timestamp', 'volume'])   
    df['date'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.sort_values(by='date').tail(20)
    print(df)


       timestamp  volume       date
3073  1727222400    1044 2024-09-25
3074  1727308800    9839 2024-09-26
3075  1727395200   19024 2024-09-27
3076  1727481600      63 2024-09-28
3077  1727568000     118 2024-09-29
3078  1727654400    6889 2024-09-30
3079  1727740800  124125 2024-10-01
3080  1727827200   52416 2024-10-02
3081  1727913600  131997 2024-10-03
3082  1728000000   13710 2024-10-04
3083  1728086400     447 2024-10-05
3084  1728172800    3216 2024-10-06
3085  1728259200   15796 2024-10-07
3086  1728345600    3691 2024-10-08
3087  1728432000    6345 2024-10-09
3088  1728518400   68432 2024-10-10
3089  1728604800    4625 2024-10-11
3090  1728691200     799 2024-10-12
3091  1728777600     159 2024-10-13
3092  1728864000   17059 2024-10-14
